# 0. Install Dependencies

In [1]:
#!pip install tensorflow==2.3.0
!pip install gym
#!pip install keras
!pip install keras-rl2

  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656447 sha256=c09d7a8a99a76d5b956a1a9a18d6cc0266d8d8ef6b21ec4eb02b17b3b6dc9586
  Stored in directory: c:\users\662748\appdata\local\pip\cache\wheels\d8\e7\68\a3f0f1b5831c9321d7523f6fd4e0d3f83f2705a1cbd5daaa79
Successfully built gym


In [19]:
import tensorflow as tf
print(tf.__version__)
!pip install tensorflow==2.4.0

2.4.0


# 1. Test Random Environment with OpenAI Gym

In [8]:
import gym 
import random

In [9]:
# Set up Environment

env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [5]:
actions

2

In [4]:
# Build Random Environment
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:23.0
Episode:2 Score:38.0
Episode:3 Score:29.0
Episode:4 Score:21.0
Episode:5 Score:13.0
Episode:6 Score:28.0
Episode:7 Score:18.0
Episode:8 Score:17.0
Episode:9 Score:40.0
Episode:10 Score:32.0


# 2. Create a Deep Learning Model with Keras

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [11]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [12]:
model = build_model(states, actions)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [13]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [14]:
model = build_model(states, actions)

In [15]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [17]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [24]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [26]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


# 4. Reloading Agent from Memory

In [ ]:
dqn = build_agent(model, actions)

In [18]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [19]:
del model
del dqn
del env

In [20]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [21]:
dqn.load_weights('dqn_weights.h5f')

In [24]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 14.000, steps: 14
Episode 2: reward: 31.000, steps: 31
Episode 3: reward: 8.000, steps: 8
Episode 4: reward: 12.000, steps: 12
Episode 5: reward: 9.000, steps: 9
Episode 6: reward: 9.000, steps: 9
Episode 7: reward: 9.000, steps: 9
Episode 8: reward: 9.000, steps: 9
Episode 9: reward: 9.000, steps: 9
Episode 10: reward: 10.000, steps: 10
Episode 11: reward: 12.000, steps: 12
Episode 12: reward: 10.000, steps: 10
Episode 13: reward: 9.000, steps: 9
Episode 14: reward: 11.000, steps: 11
Episode 15: reward: 33.000, steps: 33
